## Import Library

In [32]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import pandas_ta as ta
import pspriceaction.price_action as pa
import pandas_ta as ta

## Load Price Data

In [33]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [34]:
data = dataset.copy()

In [35]:
%%time
data = pa.pattern_modeling(data)

CPU times: user 1min 34s, sys: 517 ms, total: 1min 35s
Wall time: 1min 34s


## Calculating difference between Current close vs High & Low before

In [36]:
%%time
clone_data = data[['High', 'Low', 'Close']]
diff_high_before = []
diff_low_before = []
for i, row in clone_data.iterrows():
    if 100*row.name.hour + row.name.minute == 900:
        diff_high_before.append(0)
        diff_low_before.append(0)
    else:
        current_date = row.name.strftime('%Y-%m-%d ').format()
        current_time = row.name
        data_from_start_day = clone_data[(clone_data.index < current_time) & (clone_data.index > current_date+' 08:30:00')]
        max_high = data_from_start_day['High'].max()
        min_low = data_from_start_day['Low'].min()
        current_close = row['Close']
        dhb = 1000 * (max_high - current_close) / current_close
        dlb = 1000 * (current_close - min_low) / current_close
        diff_high_before.append(dhb)
        diff_low_before.append(dlb)
data["diff_high"] = diff_high_before
data["diff_low"] = diff_low_before
data["diff_high"] = data["diff_high"].round(1)
data["diff_low"] = data["diff_low"].round(1)

CPU times: user 57.5 s, sys: 0 ns, total: 57.5 s
Wall time: 57.5 s


In [37]:
data.dropna(inplace=True)

In [38]:
data.tail(10)

,Open,High,Low,Close,Volume,color,min_OC,max_OC,upper_wick,tail,body,model,diff_high,diff_low
Date,,,,,,,,,,,,,,
2024-07-19 13:50:00,1300.2,1302.9,1300.0,1302.9,4298,white,1300.2,1302.9,0.0,0.2,2.7,,6.4,2.9
2024-07-19 13:55:00,1303.0,1304.7,1302.7,1303.0,8426,doji,1303.0,1303.0,1.7,0.3,0.0,,6.3,3.0
2024-07-19 14:00:00,1303.1,1303.3,1301.0,1301.9,5303,black,1301.9,1303.1,0.2,0.9,1.2,,7.1,2.2
2024-07-19 14:05:00,1301.9,1301.9,1297.9,1298.0,10112,black,1298.0,1301.9,0.0,0.1,3.9,,10.2,-0.8
2024-07-19 14:10:00,1297.9,1302.5,1297.6,1301.4,11679,white,1297.9,1301.4,1.1,0.3,3.5,,7.5,2.7
2024-07-19 14:15:00,1301.2,1301.6,1298.3,1299.5,9058,black,1299.5,1301.2,0.4,1.2,1.7,,9.0,1.5
2024-07-19 14:20:00,1299.1,1301.8,1295.3,1301.6,12762,white,1299.1,1301.6,0.2,3.8,2.5,,7.4,3.1
2024-07-19 14:25:00,1302.4,1302.6,1300.1,1302.2,8096,black,1302.2,1302.4,0.2,2.1,0.2,bullish_neck,6.9,5.3
2024-07-19 14:30:00,1301.8,1301.8,1301.8,1301.8,163,doji,1301.8,1301.8,0.0,0.0,0.0,,7.2,5.0


## Strategy

In [39]:
def trade_simulation(prepared_data):
    _trades = pd.DataFrame(columns=("EntryTime", "EntryPrice", "ExitTime", "ExitPrice", "Type", "Profit", "Model"))
    has_open_deal = False
    type_open_deal = ''
    # Stoploss at x0 pips
    sl_step = 3
    # Takeprofit at y0 pips(R/R = 1/3)
    tp_step = 9
    for i, row in prepared_data.iterrows():
        if 915 < 100*row.name.hour + row.name.minute < 1420:
            if has_open_deal:
                if type_open_deal == 'short':
                    # Stoploss
                    if row['High'] > entry_price + sl_step:
                        profit = -sl_step
                        exit_price = entry_price + sl_step
                        exit_time = row.name
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit, entry_model]
                        has_open_deal = False
                        type_open_deal = ''
                    else:
                        # Takeprofit
                        if row['Low'] < entry_price - tp_step:
                            profit = tp_step
                            exit_price = entry_price - tp_step
                            exit_time = row.name
                            _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit, entry_model]
                            has_open_deal = False
                            type_open_deal = ''
                elif type_open_deal == 'long':
                    # Stoploss
                    if row['Low'] < entry_price - sl_step:
                        profit = -sl_step
                        exit_price = entry_price - sl_step
                        exit_time = row.name
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit, entry_model]
                        has_open_deal = False
                        type_open_deal = ''
                    else:
                        # Takeprofit
                        if row['High'] > entry_price + tp_step:
                            profit = tp_step
                            exit_price = entry_price + tp_step
                            exit_time = row.name
                            _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit, entry_model]
                            has_open_deal = False
                            type_open_deal = ''
                        #
            if not has_open_deal:
                if row['signal'] == 'short':
                    # Open short deal
                    entry_price = row['Close']
                    entry_time = row.name
                    entry_model = row['model']
                    has_open_deal = True
                    type_open_deal = 'short'
                elif row['signal'] == 'long':
                    # Open short deal
                    entry_price = row['Close']
                    entry_time = row.name
                    entry_model = row['model']
                    has_open_deal = True
                    type_open_deal = 'long'
        else:
            if 100*row.name.hour + row.name.minute == 1425:
                if has_open_deal:
                    # close open deal at 2:25PM (dataframe)
                    if type_open_deal == 'short':
                        profit = entry_price - row['Close']
                        if profit < -sl_step:
                            profit = -sl_step
                        elif profit > tp_step:
                            profit = tp_step
                        exit_price = row['Close']
                        exit_time = row.name
                        has_open_deal = False
                        type_open_deal = ''
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Short", profit, entry_model]
                    elif type_open_deal == 'long':
                        profit = row['Close'] - entry_price
                        if profit < -sl_step:
                            profit = -sl_step
                        elif profit > tp_step:
                            profit = tp_step
                        exit_price = row['Close']
                        exit_time = row.name
                        has_open_deal = False
                        type_open_deal = ''
                        _trades.loc[len(_trades)] = [entry_time, entry_price, exit_time, exit_price, "Long", profit, entry_model]
    return _trades

In [40]:
def has_bullish_pattern(model):
    if "bullish" in model or "rising" in model:
        return True
    return False

def has_bearish_pattern(model):
    if "bearish" in model or "falling" in model:
        return True
    return False

def cal_signal_01(signal_data):
    signal = ''
    if ("bullish" in signal_data['model'] or "rising" in signal_data['model']) and \
            (3.7 < signal_data['diff_low'] < 7.1 or signal_data['diff_low'] > 7.6) and \
            (-8.8 < signal_data['diff_high'] < 0.31 or 0.4 < signal_data['diff_high'] < 0.71 or
             1.1 < signal_data['diff_high'] < 1.61):
        signal = 'long'
    elif ("bearish" in signal_data['model'] or "falling" in signal_data['model']) and \
            (-9.3 < signal_data['diff_low'] < 2.8) and \
            (1.4 < signal_data['diff_high'] < 2.71 or 3.4 < signal_data['diff_high'] < 7.0 or
             signal_data['diff_high'] > 7.8):
        signal = 'short'
    return signal
data['signal'] = data.apply(lambda r: cal_signal_01(r), axis=1)

In [41]:
data[data.signal != '']

,Open,High,Low,Close,Volume,color,min_OC,max_OC,upper_wick,tail,body,model,diff_high,diff_low,signal
Date,,,,,,,,,,,,,,,
2018-08-13 09:55:00,946.2,946.4,945.6,946.0,1873,black,946.0,946.2,0.2,0.4,0.2,bullish_neck,0.0,3.9,long
2018-08-13 11:10:00,944.9,946.1,944.6,946.1,1643,white,944.9,946.1,0.0,0.3,1.2,fair_value_rising_gap,0.3,4.0,long
2018-08-13 11:30:00,947.1,947.3,947.1,947.3,51,white,947.1,947.3,0.0,0.0,0.2,"bullish_gap, fair_value_rising_gap",-0.3,5.3,long
2018-08-13 13:40:00,948.2,949.6,948.2,948.8,3084,white,948.2,948.8,0.8,0.0,0.6,fair_value_rising_gap,0.1,6.9,long
2018-08-14 10:40:00,953.4,953.7,952.6,952.9,2550,black,952.9,953.4,0.3,0.3,0.5,fair_value_falling_gap,4.6,-0.1,short
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-19 11:00:00,1304.3,1304.7,1303.7,1304.1,1945,black,1304.1,1304.3,0.4,0.4,0.2,falling_n,5.4,1.4,short
2024-07-19 11:10:00,1303.0,1303.0,1301.1,1302.2,6367,black,1302.2,1303.0,0.0,1.1,0.8,fair_value_falling_gap,6.9,-0.1,short
2024-07-19 13:10:00,1303.0,1303.3,1300.5,1300.7,4046,black,1300.7,1303.0,0.3,0.2,2.3,falling_three,8.1,-0.3,short


In [42]:
trade_result = trade_simulation(data)

In [43]:
total_trades = len(trade_result)
profit = trade_result.Profit.sum()
trading_fee = total_trades * 0.3
win_rate = len(trade_result[trade_result.Profit > 0]) / total_trades
display_data = {
    'trades' : total_trades,
    'profit' : profit,
    'win rate' : win_rate,
    'trading_fee' : trading_fee,
    'profit_after_fee' : profit - trading_fee
}
display_data

{'trades': 2511,
 'profit': 1969.0999999999985,
 'win rate': 0.3942652329749104,
 'trading_fee': 753.3,
 'profit_after_fee': 1215.7999999999986}

In [44]:
trade_result

,EntryTime,EntryPrice,ExitTime,ExitPrice,Type,Profit,Model
0,2018-08-13 09:55:00,946.0,2018-08-13 10:45:00,943.0,Long,-3.0,bullish_neck
1,2018-08-13 11:10:00,946.1,2018-08-13 14:25:00,954.1,Long,8.0,fair_value_rising_gap
2,2018-08-14 10:40:00,952.9,2018-08-14 13:15:00,955.9,Short,-3.0,fair_value_falling_gap
3,2018-08-14 13:15:00,956.2,2018-08-14 14:25:00,960.1,Long,3.9,fair_value_rising_gap
4,2018-08-15 10:10:00,960.1,2018-08-15 14:25:00,951.2,Short,8.9,fair_value_falling_gap
...,...,...,...,...,...,...,...
2506,2024-07-18 11:15:00,1297.5,2024-07-18 14:00:00,1288.5,Short,9.0,bearish_neck
2507,2024-07-18 14:00:00,1291.3,2024-07-18 14:15:00,1294.3,Short,-3.0,fair_value_falling_gap
2508,2024-07-19 09:20:00,1303.2,2024-07-19 09:25:00,1306.2,Short,-3.0,fair_value_falling_gap
2509,2024-07-19 09:25:00,1309.0,2024-07-19 10:20:00,1306.0,Long,-3.0,rising_three


In [45]:
trade_result_loss = trade_result[trade_result.Profit < 0]

In [46]:
trade_result_win = trade_result[trade_result.Profit > 0]

In [47]:
grouped_l1 = trade_result_loss[['Model']].groupby([trade_result_loss.Model]).count()
grouped_l1['Model'].sort_values()

Model
bearish_gap, falling_three               1
bullish_gap, rising_n                    1
bullish_separating_line                  9
rising_n                                18
falling_n                               20
rising_three                            21
bullish_gap, fair_value_rising_gap      27
bearish_gap, fair_value_falling_gap     30
bullish_gap                             36
bearish_gap                             48
falling_three                           51
bullish_neck                            76
bearish_neck                           157
fair_value_rising_gap                  411
fair_value_falling_gap                 605
Name: Model, dtype: int64

In [48]:
grouped_w1 = trade_result_win[['Model']].groupby([trade_result_win.Model]).count()
grouped_w1['Model'].sort_values()

Model
bullish_gap, rising_n                    1
bullish_separating_line                  6
falling_n                               12
bullish_gap, fair_value_rising_gap      13
rising_n                                15
bearish_gap, fair_value_falling_gap     16
rising_three                            20
bullish_gap                             22
bearish_gap                             26
falling_three                           26
bullish_neck                            54
bearish_neck                           107
fair_value_rising_gap                  307
fair_value_falling_gap                 365
Name: Model, dtype: int64

In [49]:
this_month = trade_result[(trade_result.EntryTime > '2024-06-01 00:00:00') & (trade_result.EntryTime < '2024-07-30 15:00:00')]
this_month.Profit.sum()

21.5

In [50]:
this_year = trade_result[(trade_result.EntryTime > '2024-01-01 00:00:00') & (trade_result.EntryTime < '2024-06-30 15:00:00')]
this_year.Profit.sum()

65.00000000000091

In [51]:
from datetime import date
from datetime import timedelta
from datetime import datetime
last_expire_date = date.today() - timedelta(days=90)
last_expire_date = last_expire_date.strftime("%Y-%m-%d 00:00:00")
_last_3months = trade_result[trade_result.EntryTime > last_expire_date]
_last_3months.Profit.sum()

69.40000000000055

In [52]:
_last_3months

,EntryTime,EntryPrice,ExitTime,ExitPrice,Type,Profit,Model
2413,2024-04-24 09:30:00,1214.8,2024-04-24 09:55:00,1223.8,Long,9.0,fair_value_rising_gap
2414,2024-04-24 09:55:00,1221.5,2024-04-24 10:45:00,1218.5,Long,-3.0,"bullish_gap, fair_value_rising_gap"
2415,2024-04-25 09:45:00,1226.3,2024-04-25 13:35:00,1229.3,Short,-3.0,fair_value_falling_gap
2416,2024-04-26 09:55:00,1226.8,2024-04-26 10:15:00,1235.8,Long,9.0,fair_value_rising_gap
2417,2024-04-26 10:15:00,1235.8,2024-04-26 11:20:00,1232.8,Long,-3.0,fair_value_rising_gap
...,...,...,...,...,...,...,...
2506,2024-07-18 11:15:00,1297.5,2024-07-18 14:00:00,1288.5,Short,9.0,bearish_neck
2507,2024-07-18 14:00:00,1291.3,2024-07-18 14:15:00,1294.3,Short,-3.0,fair_value_falling_gap
2508,2024-07-19 09:20:00,1303.2,2024-07-19 09:25:00,1306.2,Short,-3.0,fair_value_falling_gap
2509,2024-07-19 09:25:00,1309.0,2024-07-19 10:20:00,1306.0,Long,-3.0,rising_three


In [53]:
trade_result['EntryTime'] = trade_result['EntryTime']
k = trade_result['Profit'].groupby([trade_result.EntryTime.dt.year]).sum()
k

EntryTime
2018    125.7
2019    173.9
2020    389.2
2021    500.4
2022    484.9
2023    226.3
2024     68.7
Name: Profit, dtype: float64

In [54]:
data[(data.index > '2024-07-18 08:50:00') & (data.index < '2024-07-18 14:50:00')]

,Open,High,Low,Close,Volume,color,min_OC,max_OC,upper_wick,tail,body,model,diff_high,diff_low,signal
Date,,,,,,,,,,,,,,,
2024-07-18 09:00:00,1303.1,1304.3,1301.7,1303.9,5508,white,1303.1,1303.9,0.4,1.4,0.8,,0.0,0.0,
2024-07-18 09:05:00,1304.0,1304.0,1302.8,1303.0,1498,black,1303.0,1304.0,0.0,0.2,1.0,,1.0,1.0,
2024-07-18 09:10:00,1303.0,1303.6,1303.0,1303.5,1644,white,1303.0,1303.5,0.1,0.0,0.5,,0.6,1.4,
2024-07-18 09:15:00,1303.5,1305.0,1302.9,1303.1,4127,black,1303.1,1303.5,1.5,0.2,0.4,,0.9,1.1,
2024-07-18 09:20:00,1303.1,1303.3,1300.9,1301.2,5197,black,1301.2,1303.1,0.2,0.3,1.9,,2.9,-0.4,
2024-07-18 09:25:00,1301.3,1302.4,1300.9,1302.4,2461,white,1301.3,1302.4,0.0,0.4,1.1,,2.0,1.2,
2024-07-18 09:30:00,1302.1,1303.4,1302.1,1303.4,2915,white,1302.1,1303.4,0.0,0.0,1.3,,1.2,1.9,
2024-07-18 09:35:00,1303.3,1304.0,1302.9,1303.0,2524,black,1303.0,1303.3,0.7,0.1,0.3,,1.5,1.6,
2024-07-18 09:40:00,1303.0,1303.4,1302.6,1303.0,1412,doji,1303.0,1303.0,0.4,0.4,0.0,,1.5,1.6,
